# The StatQuest Illustrated Guide to Statistics
## Chapter 11 - Using Regression to Test for More Differences with ANOVA and ANCOVA!!!

Copyright 2026, Joshua Starmer

In this notebook we'll learn how to...

- Perform ANOVA tests using `ols()` and `anova_lm()`.
- Perform post-hoc tests to identify specific differences.
- Perform an ANCOVA test.
- Calculate *p*-values for ANOVA and ANCOVA using the null hypothesis to build a histogram.

**NOTE:**
This tutorial assumes that you have installed **[Python](https://www.python.org/)** and read Chapter 11 in **[The StatQuest Illustrated Guide to Statistics]()**.

----

Since we're using Python, the first thing we do is load in some modules that will help us do math and plot graphs.

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.api as sm
from scipy import stats
import seaborn as sns # to draw a graphs and have them look somewhat nice
import matplotlib.pyplot as plt # give us easy control of the range of values for
                                # the x and y axes.

# Performing ANOVA tests using `ols()` and `anova_lm()`

If we're going to do ANOVA, then the first thing we need is some data. In this example, we'll use the dataset illustrated in Chapter 11, which has the recovery times measured for 4 different drugs.

In [ ]:
## First, create the data
drug_a = [8, 12, 22]
drug_b = [20, 29, 39]
drug_c = [6, 17, 20]
drug_d = [45, 39, 30]

recovery_time = drug_a + drug_b + drug_c + drug_d
recovery_time

Now let's put `recovery_time` into a `data_frame` that uses a factor, `drug`, to organize each measurement. First, let's create the factor, `drug`...

In [ ]:
## create a "factor" variable that
## allows us to keep track of which drug
## is associated with which time in recovery_time
drug = (['a'] * 3) + (['b'] * 3) + (['c'] * 3) + (['d'] * 3)

## print out the values
drug

...now let's add `drug` to a `DataFrame` with `recovery_time`.

In [ ]:
## create a DataFrame
df = pd.DataFrame({
    'time': recovery_time,
    'drug': pd.Categorical(drug)
})

## print it out
df

Now, just like we did for Simple Linear Regression, Multiple Regression, and *t*-tests, we can call the `ols()` and `summary()` functions with the `DataFrame` and a simle formula, `time ~ drug`.

In [ ]:
## using OLS for ANOVA
model = smf.ols('time ~ drug', data=df)
results = model.fit()
results.summary()

The *p*-value, `Prob (F-statistic):	0.0149`, tells us to reject the Null Hypothesis that there are no differences in recovery time among the 4 drugs.

Now let's see what happens when, instead of calling `summary`, we use the `anova_lm()` function, which is specialized for summarizing ANOVA, with the same data.

In [ ]:
sm.stats.anova_lm(results)

And we see that the `anova_lm()` function simplifies the output significantly, but still gives us the same *F*-value and the same *p*-value, `PR(>F) = 0.014867`.

Now let's see how we can generate the same *p*-value using the histogram method.

In [ ]:
## since we're going to generate random datasets,
## let's start by setting the seed so that the results
## are reproducable
np.random.seed(0)

## To generate random datasets, we'll use a
## normal distribution. This distributions
## will be based on our observed data, so we
## we need to calculate the estimated
## mean and standard deviation.
overall_mean = df['time'].mean()
overall_sd = df['time'].std()

## Next, we define the number of random
## datasets we wantt o create...
num_rand_datasets = 10_000

## ...and we define the number of data points
## per dataset
num_datapoints = len(drug_a)

## Create array to store R-squared values
rand_r_squared = np.empty(num_rand_datasets)

## Here is the loop were we create a bunch of random datasets,
## each with num.datapoints values, do an ANOVA
## with the random data, then calculate and store
## the R-squared values
for i in range(num_rand_datasets):
    
    ## generate random values for each drug
    
    r_a = np.random.normal(loc=overall_mean, scale=overall_sd, size=num_datapoints)
    r_b = np.random.normal(loc=overall_mean, scale=overall_sd, size=num_datapoints)
    r_c = np.random.normal(loc=overall_mean, scale=overall_sd, size=num_datapoints)
    r_d = np.random.normal(loc=overall_mean, scale=overall_sd, size=num_datapoints)

    ## bundle the random values together in a DataFrame
    
    recovery_time_rand = np.concatenate([r_a, r_b, r_c, r_d])
    
    drug_rand = ((['drug.a'] * num_datapoints) +
                 (['drug.b'] * num_datapoints) +
                 (['drug.c'] * num_datapoints) +
                 (['drug.d'] * num_datapoints))
    
    data_rand = pd.DataFrame({'time': recovery_time_rand, 
                              'drug': pd.Categorical(drug_rand)})

    ## fit a linear regression line to the random data
    ## and calculate R-squared
    model = smf.ols('time ~ drug', data=data_rand)
    model_results = model.fit()
    rand_r_squared[i] = model_results.rsquared

Now let's draw a histogram of the $R^2$ values with the `histplot()` function...

In [ ]:
sns.histplot(data=rand_r_squared)

...and calculate the *p*-value as the precentage of "random" $R^2$ values greater than or equal to the one we got for our original data.

In [ ]:
# the number of randomly generated rsquared >= the original rsquared
num_greater = np.sum(rand_r_squared >= results.rsquared)

# calculate the p-value 
p_value = num_greater / num_rand_datasets

# print out the p-value
p_value

Thus, the *p*-value calculated with the histogram is 0.0146. Now let's compare that to the *p*-value stored in `anova.summary`...

In [ ]:
sm.stats.anova_lm(results)

So, at last, we see that the two *p*-values are essentially the same.

# BAM!

Now that we have relatively high confidence that there are differences among the four drugs, let's do all pairwise *t*-tests so that we can identify which drugs are different. These tests, done after an ANOVA to identify the indiviual groups that are different, are called **Post-Hoc** tests.

----

Performaing all pairwise *t*-tests in Python is super easy with the `posthoc_ttest()` function. Not only will this function do all of the pairwise *t*-tests for us, it will also adjust the *p*-values using any of the methods that we pass to the `multipletests()` function. We'll start by using the Holm correction for multiple testing.

NOTE: `posthoc_ttest()` is part of the `scikit_posthocs` package, and might not be installed for your system. This next chunk of code will install it if needed.

In [ ]:
%%capture
# %%capture prevents this cell from printing a ton of STDERR stuff to the screen

## First, check to see if scikit_posthocs is installed, if not, install it.
##
## NOTE: If you **do** need to install something, just know that you may need to
##       restart your session for python to find the new module(s).
##
##       To restart your session:
##       - In Google Colab, click on the "Runtime" menu and select
##         "Restart Session" from the pulldown menu
##       - In a local jupyter notebook, click on the "Kernel" menu and select
##         "Restart Kernel" from the pulldown menu
import pip
try:
  __import__("scikit_posthocs")
except ImportError:
  pip.main(['install', "scikit_posthocs"])

Now import `scikit_posthocs`...

In [ ]:
import scikit_posthocs as sp

...and now we can run `posthoc_ttest`.

In [ ]:
sp.posthoc_ttest(df, ## the DataFrame that has our data
                 val_col='time', ## column with the values
                 group_col='drug', ## column that defines how the values should be grouped
                 pool_sd=True, ## pool all data to estimate standard deviation
                 equal_var=True, ## assume variances are the same for each group.
                 p_adjust='holm') ## multiple testing correction

So, the output from `posthoc_ttest()` is a matrix of adjusted *p*-values. The first column are all the adjusted *p*-values associated with Drug A, starting with itself (Drug A) at the top and ending with Drug D at the bottom. Likewise, the second colunn gives us the adjusted *p*-values associated with Drug B , the third column gives us the adjusted *p*-values associated with Drug C, and the final column gives us the adjusted *p*-values associated with Drug D.

The last row has the two adjusted *p*-values less than 0.05 and both of them are associated with Drug D. Drug D appears to be significantly different from Drug A (adjusted *p*-value = 0.037) and Drug C (adjusted *p*-value = 0.037).

Now let's see what happens when we use False Discovery Rates (FDR) to adjust the *p*-values...

In [ ]:
sp.posthoc_ttest(df, ## the DataFrame that has our data
                 val_col='time', ## column with the values
                 group_col='drug', ## column that defines how the values should be grouped
                 pool_sd=True, ## pool all data to estimate standard deviation
                 equal_var=True, ## assume variances are the same for each group.
                 p_adjust='fdr_bh') ## multiple testing correction

...and the conclusions are the same as before. Drug D is significantly different from Drugs A and C. However, with FDR, the adjusted *p*-values tend to be a little smaller.

# Double BAM!!!

Now let's learn how to do an ANCOVA test in R.

----

# Performing an ANCOVA test

As always, we start by creating our data. In this case, we'll use the data illustrated in Chapter 11, which has recovery times for two drugs, A and B, as well as the Height of each person in the trial.

In [ ]:
## New data for drug A and B
drug_a_time = [6, 9, 12.5, 14]
drug_a_height = [4, 7.5, 10, 12.5]

drug_b_time = [4, 8, 9, 11]
drug_b_height = [5, 11, 12.5, 14]

Now let's package the data we have for the two drugs into a DataFrame.

In [ ]:
## create a factor to keep track of which
## recovery time pairs with which drug.
drug = (['Drug A'] * 4) + (['Drug B'] * 4)

df = pd.DataFrame({
    'time': drug_a_time + drug_b_time,
    'drug': pd.Categorical(drug),
    'height': drug_a_height + drug_b_height
})

## print out the data.frame
df

Now let's create a plot of the data.

In [ ]:
colors = {"Drug A": "#A5D3FC", "Drug B": "#FF968D"}

In [ ]:
my_plot= sns.scatterplot(x='height', 
                         y='time', 
                         data=df,
                         hue='drug', ## set the color based on the drug
                         s=100,
                         palette=colors,
                         edgecolor="black")

## Ensure that the legend it outside of the graph...
sns.move_legend(
    my_plot,
    "upper left", # Location within the anchor box
    bbox_to_anchor=(1, 1), # Coordinates (x, y) for the anchor box
    title="Drug"
)

## define the range of values for x and y axes.
plt.xlim(0, 16)
plt.ylim(0, 16)

Now, if we ignore the Height data, we can do a *t*-test on the two drugs, with respect to the recovery times, using the the `stats.ttest_ind()` function.

In [ ]:
stats.ttest_ind(drug_a_time, drug_b_time, equal_var=True)

The resulting *p*-value, 0.3458, is well above 0.05, so, without the Height data, we can't reject the Null Hypothesis and be confident that the two drugs are different.

Now, just for fun, let's see what happens if we ignore the Drug and just use Height to predict recovery time.

In [ ]:
model_simple = smf.ols('time ~ height', data=df)
results_simple = model_simple.fit()
results_simple.summary()

The p-value is less than 0.05, so we can reject the Null Hypothesis that using Height to predict Time is no different from using the average Time. However, this doesn't tell us if one drug is different from the other. So we need to combine both Height and Drug in a single model. This can be done by adding both column names, separated by a `+` to the **Formula** that we pass to `ols()`.

In [ ]:
## NOTE: This only compares fancy to overall mean
model_fancy = smf.ols('time ~ height + drug', data=df)
results_fancy = model_fancy.fit()
results_fancy.summary()

And when we combine both bits of information, the Drug and Height, we can predict Recovery Time much better than just using the average Recovery Time (*p*-value = 0.000184). But the Recovery Time predictions were also much better when we only used Height. So now the question is...Is using Drug and Height to predict Recovery Time better than just using Height?

One hint at the answer to this question is to compare the Adjuste R-squared values for both tests. When we compare the Adjusted R-squared values, 0.955 for Drug + Height, compared to 0.477 for just Height, we see that Adjusted R-squared goes way up, even when we add a variable to the model. The increase in Adjusted R-squared suggests that using both variables, Drug and Height, to predict Recovery Time is much better than just using Height. However, we can also calculate a *p*-value to compare the two models directly with the equation for *F*...

<span style="font-size: 24px;">
$F = \frac{[\textrm{SSR(simpler)} - \textrm{SSR(fancier)}] / (p_\textrm{fancier} - p_\textrm{simpler})}
    {\textrm{SSR(fancier)} / (n - p_\textrm{fancier})}$
</span>

...and then using *F* to calculate the area under the curve of a suitable *F*-distribution.

First, let's look at the Sum of the Squared Residuals for the simpler model that only uses Height to predict Recovery Time.

In [ ]:
results_simple.ssr

Now let's look at the Sum of the Squared Resiudals for the fancier model that uses Height and Drug to predict Recovery Times.

In [ ]:
results_fancy.ssr

Now that we have SSR(simpler) and SSR(fancier), we can calculate *F*...

In [ ]:
df1 = 1 # p_fancier - p_simpler = 3 - 2 = 1
df2 = 5 # n - p_fancier = 8 - 3 = 5

F = ((results_simple.ssr - results_fancy.ssr) / df1) / (results_fancy.ssr / df2)
F

In [ ]:
## first, import 'f'
from scipy.stats import f

## p-value with f-distribution
1-f.cdf(x=F, dfn=df1, dfd=df2)

And the resulting *p*-value tells us that using Drug and Height to predict Recovery Time is significantly different from just using Height to prediction Recovery Time. And the Adjusted R-squared we calculated earlier for the model that uses Drug and Height, 0.955, tells us that this model is much better.

Now let's see how we can calculate the *p*-value using the Histogram method.

In [ ]:
## since we're going to generate random datasets,
## let's start by setting the seed so that the results
## are reproducable
np.random.seed(0)

## To generate random datasets, we'll use a
## normal distribution. This distributions
## will be based on our observed data, so we
## we need to calculate the estimated
## mean and standard deviation.
mean_time = df['time'].mean()
sd_time = df['time'].std()

mean_height = df['height'].mean()
sd_height = df['height'].std()

## Next, we define the number of random
## datasets we wantt o create...
num_rand_datasets = 10_000

## ...and we define the number of data points
## per dataset
num_datapoints = len(drug_a_time)

## Create array to store R-squared values
rand_r_squared = np.empty(num_rand_datasets)

## Here is the loop were we create a bunch of random datasets,
## each with num_datapoints values, fit regressions to them, 
## then calculate and store the R-squared values to compare
## the residuals
for i in range(num_rand_datasets):
    
    ## generate random values for each variable
    
    r_a_time = np.random.normal(loc=mean_time, 
                                scale=sd_time, 
                                size=num_datapoints)
    r_a_height = np.random.normal(loc=mean_height, 
                                  scale=sd_height, 
                                  size=num_datapoints)
    
    r_b_time = np.random.normal(loc=mean_time, 
                                scale=sd_time, 
                                size=num_datapoints)
    r_b_height = np.random.normal(loc=mean_height, 
                                  scale=sd_height, 
                                  size=num_datapoints)

    ## bundle the random values together in a DataFrame
    
    r_time = np.concatenate([r_a_time, r_b_time])
    r_height = np.concatenate([r_a_height, r_b_height])
    
    drug_rand = ((['Drug A'] * num_datapoints) +
                 (['Drug B'] * num_datapoints))
    
    data_rand = pd.DataFrame({'time': r_time, 
                              'drug': pd.Categorical(drug_rand),
                              'height': r_height})

    simple_model = smf.ols('time ~ height', data=data_rand)
    simple_results = simple_model.fit()
    simple_ssr = simple_results.ssr
    # print("simple_ssr: " +  str(simple_ssr))


    fancy_model = smf.ols('time ~ height + drug', data=data_rand)
    fancy_results = fancy_model.fit()
    fancy_ssr = fancy_results.ssr
    # print("fancy_ssr: " + str(fancy_ssr))

    ## calculate and save the R-squared value
    rand_r_squared[i] = (simple_ssr - fancy_ssr) / simple_ssr

Now let's draw a histogram of the $R^2$ values with the `histplot()` function...

In [ ]:
sns.histplot(data=rand_r_squared)

...and calculate the *p*-value as the precentage of "random" $R^2$ values greater than or equal to the one we got for our original data.

In [ ]:
# the number of randomly generated rsquared >= the original rsquared
num_greater = np.sum(rand_r_squared >= results_fancy.rsquared)

# calculate the p-value 
p_value = num_greater / num_rand_datasets

# print out the p-value
p_value

And at last, we see that the two *p*-values are essentially the same!

# TRIPLE BAM!!!

----